# はじめに
* DeepNeuralNetを、TensorFlow等のライブラリを用いずにスクラッチで実装していく。
* ここでは全結合5層の、画像の判別機をつくる。（CNNは用いない）
* ある程度の知識は前提として進める。
* 量が多くなるため、随時更新。

# 1. 重みの初期化
* 重みを初期化するメソッドを作成する
* DeepLearningで学習を進める前に、W,bを初期化する必要がある。
* Wは0で初期化すると、学習が進まない。
* 最適な初期値については、Xavier,Heの初期値、といったものがあるが、ここでは単純に微小なランダムの値とする。

In [1]:
import numpy as np
np.random.seed(1)

In [2]:
def initialize_parameters(layer_dims):
    parameters = {}
    L = len(layer_dims)
    
    for l in range(1,L):
        parameters["W" + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1]) * 0.01
        parameters["b" + str(l)] = np.zeros((layer_dims[l],1))
        
    return parameters

* 上記は`layer_dims`に作成するDLの各層のユニット数を指定することで、Wとbを返してくれる。
* 例えば、inputは5、outputは1、隠れ層1層でユニット数3とすると下記。

In [3]:
initialize_parameters([5,3,1])

{'W1': array([[ 0.01624345, -0.00611756, -0.00528172, -0.01072969,  0.00865408],
        [-0.02301539,  0.01744812, -0.00761207,  0.00319039, -0.0024937 ],
        [ 0.01462108, -0.02060141, -0.00322417, -0.00384054,  0.01133769]]),
 'W2': array([[-0.01099891, -0.00172428, -0.00877858]]),
 'b1': array([[ 0.],
        [ 0.],
        [ 0.]]),
 'b2': array([[ 0.]])}

* W1,W2,b1,b2を作成してくれる。

# 2. Forward Propagation
* Forward Propagationを実装する
* まずは1つ前のActivation Functionの返り値とW,bを基に、活性化関数に入れる数値を計算。

In [4]:
def linear_forward(A,W,b):
    Z = np.dot(W,A) + b
    
    # Backwardを計算する際に用いる
    cache = (A,W,b)
    
    return Z, cache

* 活性化関数を作成する。
* ReLU関数とsigmoid関数をつくる。

In [5]:
def relu(Z):
    A = Z * (Z > 0)
    return A

In [6]:
def sigmoid(Z):
    A = 1 / (1 + np.exp(Z))
    return A

* ここまでに作成した、ReLU,sigmoid,linear_forwardを元に、前層のActicationの返り値から次のActivationの返り値を返す関数を作る

In [7]:
def linear_activation_forward(A_prev, W, b,activation):

    Z, linear_cache = linear_forward(A_prev,W,b)

    if activation == "relu":
        A, activation_cache = relu(Z)
    
    if activation == "sigmoid":
        A, activation_cache = sigmoid(Z)
    
    cache = (linear_cache, activation_cache)
    
    return A, cache

* ここまでで各層での計算の関数はできたため、最後にinputからoutputを返す関数を作成する

In [8]:
def model_forward(X, parameters):
    caches = []
    A = X
    # 各層の間にW,bがあるため、数を2で割れば層数になる
    L = len(parameters) // 2 
    
    # 隠れ層はReLUを用いる
    for l in range(L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters["W"+str(l)],parameters["b"+str(l)],"relu")
        caches.append(cache)
    
    # 最終層はSigmoidを用いる
    AL, cache = linear_activation_forward(A, parameters["W"+str(L)], parameters["b"+str(L)],"sigmoid")
    caches.append(cache)
    
    return AL, caches

* 上記でForwardPropagationは実装できた

# 3. Cost Function
* ForwardPropagationによって、あるinputに対し、あるパラメータの時の、予測値を求めることができるようになった。
* ここではその予測結果がどれだけ正しいのかを算出するために、Cost関数を作成する。
* クロスエントロピー誤差を用いる。

In [9]:
def compute_cost(AL, Y):
    m = Y.shape
    
    cost = - 1/m * (np.dot(Y, np.log(AL.T)) + np.dot(1-Y, np.log((1-AL).T)))
    
    # costのshapeを変更して、数値型で返すようにする。
    cost = np.squeeze(cost)
    
    return cost

* np.squeezeは下記のようなことをしてくれている

In [10]:
np.squeeze([[[[[13]]]]])

array(13)

* 続きます

## 参考
* [Coursera deeplearning.ai](https://www.coursera.org/learn/neural-networks-deep-learning/home/welcome)